### Reprojection: transforming map DataArrays between coordinate reference systems (crs)
#### Xarray-spatial's reproject takes in a spatially referenced source raster aligned with one coordinate reference system (crs) and transforms it into another crs, warping the shape and features and resetting the coordinates.

Let's bring in the packages we'll be using. First, we'll import the basic data manipulation packages, Numpy and Xarray, and then Rasterio and Rioxarray, two packages important for dealing with rasters and crs's.  

Then we'll get Geopandas, Spatialpandas, and some geometry classes for dealing with shapefile data and geometries.  

We'll also want datashader and some of its rendering functions for easy visualization of our generated data.

Finally, we'll import Xarray-spatial's reproject.

In [ ]:
import numpy as np
import xarray as xr

import rasterio
import rioxarray

import geopandas as gpd
import spatialpandas as spd
from shapely.geometry.point import Point
from shapely.geometry.linestring import LineString

import datashader as ds
from datashader.transfer_functions import shade, stack, set_background
from datashader.colors import Elevation

from reprojection import reproject

To get started, let's set up a shader function to render our output DataArrays overlayed with the target country shapes for the desired crs.

In [ ]:
# first set up countries pts GeoDataFrame to use for crs transformations
def setup_countries_pts():
    world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
    countries = world[['name', 'geometry']]
    countries_src_crs = countries.crs
    
    countries = countries.explode('geometry').reset_index(drop=True)
    
    points_dict = {'geometry': [], 'grp_idx': []}
    for i in range(len(countries['geometry'].values)):
        line = countries['geometry'].values[i]
        list_coords = list(line.array_interface_base['data'])
        for j in range(0, len(list_coords), 2):
            x_coord = list_coords[j]
            y_coord = list_coords[j + 1]
            point = Point(x_coord, y_coord)
            points_dict['geometry'].append(point)
            points_dict['grp_idx'].append(i)
    
    points_gdf = gpd.GeoDataFrame(points_dict, geometry='geometry', crs=countries_src_crs)
    return points_gdf


country_pts_gdf = setup_countries_pts()


def countries_warp_boundary(points, dst_crs):
    points_gdf = points.to_crs(dst_crs)
    points_gdf_lines = points_gdf.groupby('grp_idx')
    pts_lines_geom = points_gdf_lines['geometry']
    lines_dict = {'geometry': []}
    for pts in pts_lines_geom:
        pts_set = pts[1]
        pts_list = [pt for pt in pts_set]
        line = LineString(pts_list)
        lines_dict['geometry'].append(line)

    countries_lines = gpd.GeoDataFrame(lines_dict, geometry='geometry')
    return countries_lines


def stack_countries_elevation(elevation):
    elev_crs = elevation.rio.crs
    if len(elevation.shape) == 3:
        elevation = elevation.squeeze()
    
    elev_height, elev_width = elevation.shape
    
    country_lines = countries_warp_boundary(country_pts_gdf, elev_crs)
    
    countries_spd = spd.GeoDataFrame(country_lines, geometry='geometry')

    minx, maxx = countries_spd['geometry'].bounds['x0'].min(),\
                 countries_spd['geometry'].bounds['x1'].max()
    miny, maxy = countries_spd['geometry'].bounds['y0'].min(),\
                 countries_spd['geometry'].bounds['y1'].max()

    canvas = ds.Canvas(plot_height=elev_height, plot_width=elev_width, x_range=(minx, maxx), y_range=(miny, maxy))
    
    elev_rst = canvas.raster(elevation)
    shade_elev = shade(elev_rst, cmap=Elevation, min_alpha=0, alpha=165)
    
    countries_rst = canvas.line(countries_spd, geometry='geometry')
    countries_rst.coords['x'] = elev_rst.coords['x']
    countries_rst.coords['y'] = elev_rst.coords['y']
    countries_rst.data = ds.utils.orient_array(countries_rst)
    shade_countries = shade(countries_rst, cmap=['yellow'], alpha=255)
#     return shade_countries
    return stack(shade_countries, shade_elev)

Now, we're ready to generate some map images.

Representing a 3D surface like the earth in a 2D map has been a challenge for a long time in cartography. Many solutions have been found, but ultimately it comes down to a compromise between distortions in distance, direction, and area. Different coordinate reference systems (crs) perform differently in these 3 metrics in different map areas.

A few of the more well-known projections follow:

The lat/lon (equirectangular) projection, EPSG:4326:  
This very common one displays meridians and parallels as equally spaced vertical and horizontal lines. Conversion between x, y coordinates on the map and earth locations are very simple, and this is a very popular one for general use. However, the significant distortions inherent to this method make it impractical for many scientific application.

Let's open an earth elevation map in this crs.

In [ ]:
earth_lat_lon = rioxarray.open_rasterio('/Users/ls/Downloads/elevation.tif')
stack_countries_elevation(earth_lat_lon)

The shapes and relative sizes of the countries look like what we'd expect with this crs, as you can confirm with the neat yellow outlines around the countries.

Now, we'll use Xarray-spatial's `reproject` to reproject this map into the web mercator crs. Web mercator is a cylindrical projection and produces a conformal map (true angles), and it is the projection of choice for almost all web maps.

To perform the reprojection, we just need the EPSG code, 3857. 

In [ ]:
web_mercator_crs = 'EPSG:3857'
web_mercator_earth = reproject(earth_lat_lon, web_mercator_crs)
stack_countries_elevation(web_mercator_earth)

As you can see, this shrinks the sizes of countries closer to equator and stretches those at the poles in order to preserve the angles between directions.
As you can also see, the yellow country oulines neatly wrap the countries displayed, showing that our reprojection is returning the correct shapes. Please note the cutoffs at the top and bottom of this map; these are due to the web mercator projection not applying to latitudes above and below +/-85.06 degrees.

Now, let's try the Robinson projection. This map was designed to be "right appearing" and have a more intuitive feel to human readers, and has small distorions in all dimensions. It's code is ESRI:54030.

In [ ]:
robinson_crs = 'ESRI:54030'
robinson_earth = reproject(earth_lat_lon, robinson_crs)
stack_countries_elevation(robinson_earth)

Note the 'rounded' perspective given to the map and countries, giving the appearance of viewing a spherical map, and giving room for intuitive interpretation and comprehension of relative shapes and directions. 

Another well-used one is the Transverse Mercator projection. Like the web mercator projection it's cylindrical and conformal, but the axes are switched; i.e. the earth sphere is stretched towards two parallels. It's most useful areas with small longitude ranges, such as states and counties.

In [ ]:
tvs_mctr_Italy = 'EPSG:3004'
transverse_mercator_earth = reproject(earth_lat_lon, tvs_mctr_Italy)
stack_countries_elevation(transverse_mercator_earth)

The Lambert Conformal Conic projection, which works best around middle meridians.

In [ ]:
lambert_NA_crs = 'ESRI:102009'
lambert_earth = reproject(earth_lat_lon, lambert_NA_crs)
stack_countries_elevation(lambert_earth)

Another, unique, projection is the Space Oblique. This is a projection especially designed for satellite imagery so that it is completely free of distortion along the orbit path of the satellite.

In [ ]:
space_oblique_crs = 'EPSG:29873'
space_oblique_earth = reproject(earth_lat_lon, space_oblique_crs)
stack_countries_elevation(space_oblique_earth)